# Introduction to Project

This is credit modeling project. Fit and predict one's income based on various indicators

## Importing the dataset
[Dataset Here](https://datahub.io/machine-learning/adult#resource-adult)
[About the Dataset](https://archive.ics.uci.edu/ml/datasets/Adult)
Please store in the same working director as your jupyter notebook.


In [93]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

## Columns/Features of Dataset

In [95]:
# import the file path and import csv to dataframe
file = 'adult.csv'
df = pd.read_csv(file)

# get the column names and count
columns = df.columns; columns_list= columns.tolist()
num_columns = columns.nunique(); print("Number of columns:", num_columns, "\n", columns_list)

Number of columns: 15 
 ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capitalgain', 'capitalloss', 'hoursperweek', 'native-country', 'class']


In [96]:
# preview the data fields
preview = df.head(1); print(preview, "\n")

   age  workclass  fnlwgt  education  education-num marital-status  \
0    2  State-gov   77516  Bachelors             13  Never-married   

     occupation   relationship   race   sex  capitalgain  capitalloss  \
0  Adm-clerical  Not-in-family  White  Male            1            0   

   hoursperweek native-country  class  
0             2  United-States  <=50K   



## Dataset Metadata

In [97]:
# the data type of each of the columns
variable_types = df.dtypes.to_dict()
print(variable_types)

{'age': dtype('int64'), 'workclass': dtype('O'), 'fnlwgt': dtype('int64'), 'education': dtype('O'), 'education-num': dtype('int64'), 'marital-status': dtype('O'), 'occupation': dtype('O'), 'relationship': dtype('O'), 'race': dtype('O'), 'sex': dtype('O'), 'capitalgain': dtype('int64'), 'capitalloss': dtype('int64'), 'hoursperweek': dtype('int64'), 'native-country': dtype('O'), 'class': dtype('O')}


# GOAL: Predict whether income exceeds $50K/yr based on census data.

## Column Overviews

In [99]:
for x in columns_list:
    print(x) # print column name
    variable_categories = df[x].unique(); print(variable_categories, "\n")

age
[2 3 1 0 4] 

workclass
['State-gov' 'Self-emp-not-inc' 'Private' 'Federal-gov' 'Local-gov' nan
 'Self-emp-inc' 'Without-pay' 'Never-worked'] 

fnlwgt
[ 77516  83311 215646 ... 173449  89686 350977] 

education
['Bachelors' 'HS-grad' '11th' 'Masters' '9th' 'Some-college' 'Assoc-acdm'
 'Assoc-voc' '7th-8th' 'Doctorate' 'Prof-school' '5th-6th' '10th'
 '1st-4th' 'Preschool' '12th'] 

education-num
[13  9  7 14  5 10 12 11  4 16 15  3  6  2  1  8] 

marital-status
['Never-married' 'Married-civ-spouse' 'Divorced' 'Married-spouse-absent'
 'Separated' 'Married-AF-spouse' 'Widowed'] 

occupation
['Adm-clerical' 'Exec-managerial' 'Handlers-cleaners' 'Prof-specialty'
 'Other-service' 'Sales' 'Craft-repair' 'Transport-moving'
 'Farming-fishing' 'Machine-op-inspct' 'Tech-support' nan
 'Protective-serv' 'Armed-Forces' 'Priv-house-serv'] 

relationship
['Not-in-family' 'Husband' 'Wife' 'Own-child' 'Unmarried' 'Other-relative'] 

race
['White' 'Black' 'Asian-Pac-Islander' 'Amer-Indian-Eskimo' 'Ot

# Mapping Categorical Variables to Numeric
String variables will have to be mapped into a numeric character using the <code>df.replace([old list],[new list])</code>